# 1. Linkování C knihovny do Pythonu
V této části připravíme C funkci, kterou budeme v dalších noteboocích volat přes `ctypes`, `cffi`, Cython a nativní C API (Application Programming Interface).


## 1.1 C signatura a ABI
Použijeme tuto signaturu. ABI (Application Binary Interface) je binární dohoda mezi volající a volanou stranou: typy argumentů, jejich pořadí, návratová hodnota a konvence volání musí sedět přesně.

- `const float *matrix`, `const float *vector`: vstupní data pouze pro čtení,
- `float *result`: výstupní buffer, do kterého C zapisuje,
- `size_t rows`, `size_t cols`: nezáporné rozměry (šířka odpovídá platformě).

Matice je v paměti po řádcích (C-order), tedy index `i * cols + j`.

Poznámka k přenositelnosti: `long` není mezi platformami stabilní (`Linux x86_64` typicky 64 bit, `Windows x64` typicky 32 bit), proto pro rozměry používáme `size_t`.


In [ ]:
%%writefile matrix_vector_multiply.h
#include <stddef.h>

void matrix_vector_multiply(
    const float *matrix,
    const float *vector,
    float *result,
    size_t rows,
    size_t cols
);


In [ ]:
%%writefile matrix_vector_multiply.c
#include "matrix_vector_multiply.h"

void matrix_vector_multiply(
    const float *matrix,
    const float *vector,
    float *result,
    size_t rows,
    size_t cols
) {
    for (size_t i = 0; i < rows; ++i) {
        float acc = 0.0f;
        for (size_t j = 0; j < cols; ++j) {
            acc += matrix[i * cols + j] * vector[j];
        }
        result[i] = acc;
    }
}


## 1.2 Kompilace sdílené knihovny
Na Linuxu vytvoříme `.so` takto:

```bash
gcc -shared -fPIC -O3 -Wall -Wextra -std=c11 -o libmatrix_vector_multiply.so matrix_vector_multiply.c
```

Co znamenají použité přepínače:
- `-shared`: vytvoří sdílenou knihovnu místo spustitelného programu,
- `-fPIC`: generuje position-independent code (nutné pro bezpečné načítání sdílené knihovny),
- `-O3`: vyšší úroveň optimalizací,
- `-Wall` a `-Wextra`: zapnou širší sadu varování překladače,
- `-std=c11`: překládá podle standardu C11,
- `-o ...`: jméno výstupního souboru.

Na Windows je výstup obvykle `.dll` a při načítání přes `ctypes` se často používá `WinDLL`.


In [ ]:
!gcc -shared -fPIC -O3 -Wall -Wextra -std=c11 -o libmatrix_vector_multiply.so matrix_vector_multiply.c